In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()

mtop = np.ones([mfmodel.disv.nvert.array ])*mfmodel.disv.top[0]
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0]

flopy is installed in /home/gida2/.local/lib/python3.6/site-packages/flopy
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:
mfmodel.disv.cell2d

{internal}
(rec.array([(    0, 624365.32 , 8349269.99, 7, 64439, 64291, 64707, 68556, 69977, 64671, 64439, None, None, None, None, None),
           (    1, 624875.935, 8349288.08, 8, 73223, 74750, 74033, 73401, 69977, 68556, 69385, 73223, None, None, None, None),
           (    2, 625361.448, 8351451.3 , 8, 76972, 76308, 74104, 73982, 74126, 76734, 76860, 76972, None, None, None, None),
           ...,
           (49523, 629303.926, 8369860.63, 8, 91079, 91080, 91127, 91148, 91099, 91093, 91076, 91079, None, None, None, None),
           (49524, 629283.165, 8369860.94, 5, 91025, 91079, 91076, 91039, 91025, None, None, None, None, None, None, None),
           (49525, 629262.317, 8369859.4 , 7, 91025, 91039, 91037, 90965, 90972, 91015, 91025, None, None, None, None, None)],
          dtype=[('icell2d', '<i8'), ('xc', '<f8'), ('yc', '<f8'), ('ncvert', '<i8'), ('icvert_0', '<i8'), ('icvert_1', '<i8'), ('icvert_2', '<i8'), ('icvert_3', '<i8'), ('icvert_4', 'O'), ('icvert_5', 'O'), ('icve

In [3]:
modelz = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array))
nlay = mfmodel.disv.nlay.array

0
((0, 351), 3923., 1.5)
1
((0, 721), 3799., 1.5)
2
((0, 725), 3927., 1.5)
3
((0, 1390), 3799., 1.5)
4
((0, 1679), 4437., 1.5)


In [8]:
drnPkg = mfmodel.get_package('drn_0')
drainTubes = {}
for index, drnCell in enumerate(drnPkg.stress_period_data.array[0]):
    cellNumber = drnCell[0][1]
    cellLay = drnCell[0][0]
    cellGrid = mfmodel.disv.cell2d.array[cellNumber]
    vertexIndexList = [x for x in list(cellGrid)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelz[cellLay,cellNumber]] for vertex in vertexIndexList])
    cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
    cellSurf.cell_arrays['elevation'] = drnCell[1]
    cellSurf.cell_arrays['cond'] = drnCell[2]
    cellZ = modelz[:,cellNumber]
    cellMesh = cellSurf.extrude([0, 0, cellZ[cellLay +1]-cellZ[cellLay]])
    drainTubes[str(index)] = cellMesh 
totalDrainsCells = pv.MultiBlock(drainTubes)
totalDrainsMerged = totalDrainsCells.combine()
totalDrainsMerged.save('../vtk/modelDrains.vtk')